# Modelo Auto ARIMA (AutoRegressive Integrated Moving Average Automático)

Este notebook implementa modelos Auto ARIMA para predecir las ventas de dos productos, utilizando dos tipos de validación temporal:
- Rolling Window
- Expanding Window

**Características del Auto ARIMA:**
- Encuentra automáticamente los mejores parámetros (p, d, q) usando búsqueda exhaustiva
- Utiliza AIC (Akaike Information Criterion) como criterio de selección
- Aplicado a ambos productos

Después de validar con los dos métodos, se selecciona el mejor método basado en el RMSE promedio más bajo y se entrena el modelo final para producción. Posteriormente se validan los supuestos estadísticos del modelo.


In [1]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch
from statsmodels.tsa.stattools import adfuller
from scipy import stats
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', ConvergenceWarning)


In [2]:
# Cargar el dataset
df = pd.read_csv('data-set.csv', index_col=0)
print("Dataset cargado:")
print(f"Forma del dataset: {df.shape}")
print(f"\nPrimeras filas:")
print(df.head())
print(f"\nÚltimas filas:")
print(df.tail())
print(f"\nInformación del dataset:")
print(df.info())


Dataset cargado:
Forma del dataset: (127, 2)

Primeras filas:
    producto1   producto2
1  500.000000  200.000000
2  497.400893  210.686220
3  478.605317  222.018584
4  486.454125  233.920990
5  479.695678  238.402098

Últimas filas:
      producto1   producto2
123  164.610771  629.293034
124  150.881839  637.099467
125  151.788470  653.155282
126  137.047639  672.528345
127  141.990873  676.058092

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 1 to 127
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   producto1  127 non-null    float64
 1   producto2  127 non-null    float64
dtypes: float64(2)
memory usage: 3.0 KB
None


## Función Auto ARIMA

Implementación del modelo Auto ARIMA que encuentra automáticamente los mejores órdenes (p, d, q):
- **p**: Orden del componente autorregresivo (AR)
- **d**: Orden de diferenciación
- **q**: Orden del componente de media móvil (MA)

El modelo Auto ARIMA busca automáticamente la mejor combinación de parámetros dentro de los rangos especificados.


In [3]:
def auto_arima_forecast(data, max_p=5, max_d=3, max_q=5, seasonal=False, forecast_horizon=1, suppress_warnings=True):
    """
    Calcula el modelo Auto ARIMA y realiza predicciones.
    Busca automáticamente los mejores parámetros (p, d, q) usando AIC como criterio.
    
    Parameters:
    -----------
    data : array-like
        Serie temporal de datos
    max_p : int
        Valor máximo para el orden p (AR)
    max_d : int
        Valor máximo para el orden d (diferenciación)
    max_q : int
        Valor máximo para el orden q (MA)
    seasonal : bool
        Si se debe considerar estacionalidad (no implementado en esta versión)
    forecast_horizon : int
        Número de períodos a predecir (por defecto 1)
    suppress_warnings : bool
        Suprimir advertencias durante el ajuste
    
    Returns:
    --------
    float or array
        Predicción(es) usando Auto ARIMA
    """
    if len(data) < 2:
        return data[-1] if len(data) > 0 else 0
    
    # Convertir a pandas Series
    data_series = pd.Series(data)
    
    try:
        # Búsqueda automática de mejores parámetros usando AIC
        best_aic = np.inf
        best_order = None
        best_model = None
        
        # Buscar en el espacio de parámetros
        for p in range(0, max_p + 1):
            for d in range(0, max_d + 1):
                for q in range(0, max_q + 1):
                    try:
                        # Intentar ajustar el modelo
                        model = ARIMA(data_series, order=(p, d, q))
                        fitted_model = model.fit()
                        
                        # Usar AIC como criterio de selección
                        if fitted_model.aic < best_aic:
                            best_aic = fitted_model.aic
                            best_order = (p, d, q)
                            best_model = fitted_model
                    except:
                        # Si el modelo no converge, continuar con el siguiente
                        continue
        
        # Si no se encontró ningún modelo válido, usar un modelo simple
        if best_model is None:
            # Intentar con un modelo simple
            try:
                model = ARIMA(data_series, order=(1, 1, 1))
                best_model = model.fit()
            except:
                # Si falla, usar el último valor
                return data[-1]
        
        # Realizar predicción
        forecast = best_model.forecast(steps=forecast_horizon)
        
        # Si solo se predice un paso, devolver un escalar
        if forecast_horizon == 1:
            return float(forecast.iloc[0]) if hasattr(forecast, 'iloc') else float(forecast[0])
        else:
            # Devolver array de predicciones
            if hasattr(forecast, 'values'):
                return forecast.values
            else:
                return np.array(forecast)
        
    except Exception as e:
        # En caso de error, usar el último valor como predicción
        if not suppress_warnings:
            print(f"Advertencia en Auto ARIMA: {e}")
        return data[-1]


## Función 1: Walk-Forward Validation
    
En Walk-Forward Validation, se entrena el modelo con datos hasta un punto específico y se valida con el siguiente punto. Luego se avanza un paso y se repite el proceso.


In [4]:
def walk_forward_validation(dataset, max_p=5, max_d=3, max_q=5, seasonal=False, train_size_min=24):
    """
    Realiza validación Walk-Forward para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    max_p : int
        Valor máximo para el orden p (AR)
    max_d : int
        Valor máximo para el orden d (diferenciación)
    max_q : int
        Valor máximo para el orden q (MA)
    seasonal : bool
        Si se debe considerar estacionalidad
    train_size_min : int
        Tamaño mínimo de entrenamiento antes de comenzar la validación (por defecto 24)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Walk-Forward Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = auto_arima_forecast(
                    train_data, 
                    max_p=max_p,
                    max_d=max_d,
                    max_q=max_q,
                    seasonal=seasonal,
                    forecast_horizon=1,
                    suppress_warnings=True
                )
            except Exception as e:
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 2: Rolling Window Validation

En Rolling Window Validation, se utiliza una ventana fija de tamaño constante que se desliza a lo largo de la serie temporal.


In [5]:
def rolling_window_validation(dataset, max_p=5, max_d=3, max_q=5, seasonal=False, train_window_size=24):
    """
    Realiza validación Rolling Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    max_p : int
        Valor máximo para el orden p (AR)
    max_d : int
        Valor máximo para el orden d (diferenciación)
    max_q : int
        Valor máximo para el orden q (MA)
    seasonal : bool
        Si se debe considerar estacionalidad
    train_window_size : int
        Tamaño fijo de la ventana de entrenamiento (por defecto 24)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Rolling Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar desde train_window_size hasta n-1
        for i in range(train_window_size, n):
            # Datos de entrenamiento: ventana fija de tamaño train_window_size
            train_data = data[i - train_window_size:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = auto_arima_forecast(
                    train_data, 
                    max_p=max_p,
                    max_d=max_d,
                    max_q=max_q,
                    seasonal=seasonal,
                    forecast_horizon=1,
                    suppress_warnings=True
                )
            except Exception as e:
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_window_size + 1,
                'ventana': f"{i - train_window_size}-{i}",
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_window_size + 1) % 20 == 0:
                print(f"  Iteración {i - train_window_size + 1}/{n - train_window_size}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 3: Expanding Window Validation

En Expanding Window Validation, la ventana de entrenamiento crece con cada iteración, comenzando desde un tamaño mínimo y expandiéndose hasta incluir todos los datos disponibles hasta ese punto.


In [6]:
def expanding_window_validation(dataset, max_p=5, max_d=3, max_q=5, seasonal=False, train_size_min=24):
    """
    Realiza validación Expanding Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    max_p : int
        Valor máximo para el orden p (AR)
    max_d : int
        Valor máximo para el orden d (diferenciación)
    max_q : int
        Valor máximo para el orden q (MA)
    seasonal : bool
        Si se debe considerar estacionalidad
    train_size_min : int
        Tamaño mínimo inicial de la ventana de entrenamiento (por defecto 24)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Expanding Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i (ventana que se expande)
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = auto_arima_forecast(
                    train_data, 
                    max_p=max_p,
                    max_d=max_d,
                    max_q=max_q,
                    seasonal=seasonal,
                    forecast_horizon=1,
                    suppress_warnings=True
                )
            except Exception as e:
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,  # Tamaño de la ventana que se expande
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Ejecución de las Validaciones

Después de definir las funciones que permitirán ejecutar el modelo, ahora ejecutamos las validaciones para cada producto usando **Rolling Window** y **Expanding Window** (Walk-Forward Validation está disponible pero no se utiliza en este análisis) y calculamos las métricas promedio.

**Configuración inicial del Auto ARIMA:**
- Valores por defecto: max_p=5, max_d=3, max_q=5
- Sin estacionalidad inicialmente
- El Auto ARIMA buscará automáticamente los mejores parámetros (p, d, q) dentro de estos rangos usando AIC como criterio


In [7]:
# Parámetros de validación
TRAIN_SIZE_MIN = 24  # Tamaño mínimo de entrenamiento
TRAIN_WINDOW_SIZE = 24  # Tamaño fijo para Rolling Window

# Parámetros iniciales del Auto ARIMA (se optimizarán después)
MAX_P_INIT = 5
MAX_D_INIT = 3
MAX_Q_INIT = 5
SEASONAL_INIT = False

print("=" * 60)
print("CONFIGURACIÓN INICIAL DEL AUTO ARIMA")
print("=" * 60)
print(f"max_p inicial: {MAX_P_INIT}")
print(f"max_d inicial: {MAX_D_INIT}")
print(f"max_q inicial: {MAX_Q_INIT}")
print(f"Estacionalidad: {SEASONAL_INIT}")
print(f"Tamaño mínimo de entrenamiento: {TRAIN_SIZE_MIN}")
print(f"Tamaño de ventana fija (Rolling): {TRAIN_WINDOW_SIZE}")


CONFIGURACIÓN INICIAL DEL AUTO ARIMA
max_p inicial: 5
max_d inicial: 3
max_q inicial: 5
Estacionalidad: False
Tamaño mínimo de entrenamiento: 24
Tamaño de ventana fija (Rolling): 24


In [8]:
# NOTA: Walk-Forward Validation está disponible pero no se utiliza en este análisis, ya que tipo de validación es similar
# a el Expanding Window Validation
# Se ejecutan solo Rolling Window y Expanding Window Validation
print("\n" + "=" * 60)
print("NOTA: Walk-Forward Validation disponible pero no utilizada")
print("=" * 60)
print("Se ejecutarán solo Rolling Window y Expanding Window Validation")



NOTA: Walk-Forward Validation disponible pero no utilizada
Se ejecutarán solo Rolling Window y Expanding Window Validation


In [9]:
# Ejecutar Rolling Window Validation para cada producto
print("\n" + "=" * 60)
print("ROLLING WINDOW VALIDATION")
print("=" * 60)

results_rw = rolling_window_validation(
    df, 
    max_p=MAX_P_INIT,
    max_d=MAX_D_INIT,
    max_q=MAX_Q_INIT,
    seasonal=SEASONAL_INIT,
    train_window_size=TRAIN_WINDOW_SIZE
)



ROLLING WINDOW VALIDATION

=== Rolling Window Validation para producto1 ===
  Iteración 20/103: RMSE=869.4679, MAE=869.4679
  Iteración 40/103: RMSE=1474.5778, MAE=1474.5778
  Iteración 60/103: RMSE=5092.8912, MAE=5092.8912
  Iteración 80/103: RMSE=1664.1148, MAE=1664.1148
  Iteración 100/103: RMSE=1838.4129, MAE=1838.4129
  Total de iteraciones: 103

=== Rolling Window Validation para producto2 ===
  Iteración 20/103: RMSE=6079.7469, MAE=6079.7469
  Iteración 40/103: RMSE=9234.5174, MAE=9234.5174
  Iteración 60/103: RMSE=7136.5504, MAE=7136.5504
  Iteración 80/103: RMSE=12170.8931, MAE=12170.8931
  Iteración 100/103: RMSE=223.9764, MAE=223.9764
  Total de iteraciones: 103


In [10]:
# Ejecutar Expanding Window Validation para cada producto
print("\n" + "=" * 60)
print("EXPANDING WINDOW VALIDATION")
print("=" * 60)

results_ew = expanding_window_validation(
    df, 
    max_p=MAX_P_INIT,
    max_d=MAX_D_INIT,
    max_q=MAX_Q_INIT,
    seasonal=SEASONAL_INIT,
    train_size_min=TRAIN_SIZE_MIN
)



EXPANDING WINDOW VALIDATION

=== Expanding Window Validation para producto1 ===
  Iteración 20/103: RMSE=3.3639, MAE=3.3639
  Iteración 40/103: RMSE=6.0466, MAE=6.0466
  Iteración 60/103: RMSE=14.8514, MAE=14.8514
  Iteración 80/103: RMSE=0.2209, MAE=0.2209
  Iteración 100/103: RMSE=9.6959, MAE=9.6959
  Total de iteraciones: 103

=== Expanding Window Validation para producto2 ===
  Iteración 20/103: RMSE=3.0548, MAE=3.0548
  Iteración 40/103: RMSE=3.5761, MAE=3.5761
  Iteración 60/103: RMSE=5.0500, MAE=5.0500
  Iteración 80/103: RMSE=6.2551, MAE=6.2551
  Iteración 100/103: RMSE=1.9653, MAE=1.9653
  Total de iteraciones: 103


In [11]:
def calcular_metricas_promedio(results, nombre_validacion):
    """
    Calcula las métricas promedio para cada producto.
    
    Parameters:
    -----------
    results : dict
        Resultados de la validación
    nombre_validacion : str
        Nombre del tipo de validación
    
    Returns:
    --------
    dict
        Diccionario con métricas promedio por producto
    """
    metricas_promedio = {}
    
    print(f"\n{'='*60}")
    print(f"MÉTRICAS PROMEDIO - {nombre_validacion.upper()}")
    print(f"{'='*60}")
    
    for producto in results.keys():
        metrics_list = results[producto]
        
        # Calcular promedios
        rmse_promedio = np.mean([m['rmse'] for m in metrics_list])
        mae_promedio = np.mean([m['mae'] for m in metrics_list])
        
        metricas_promedio[producto] = {
            'rmse_promedio': rmse_promedio,
            'mae_promedio': mae_promedio,
            'num_iteraciones': len(metrics_list)
        }
        
        print(f"\n{producto}:")
        print(f"  RMSE Promedio: {rmse_promedio:.4f}")
        print(f"  MAE Promedio: {mae_promedio:.4f}")
        print(f"  Número de iteraciones: {len(metrics_list)}")
    
    return metricas_promedio

# Calcular métricas promedio para cada tipo de validación
# NOTA: Walk-Forward no se utiliza, solo se calculan métricas para Rolling Window y Expanding Window
metricas_rw = calcular_metricas_promedio(results_rw, "Rolling Window")
metricas_ew = calcular_metricas_promedio(results_ew, "Expanding Window")



MÉTRICAS PROMEDIO - ROLLING WINDOW

producto1:
  RMSE Promedio: 2589.2507
  MAE Promedio: 2589.2507
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 6018.4468
  MAE Promedio: 6018.4468
  Número de iteraciones: 103

MÉTRICAS PROMEDIO - EXPANDING WINDOW

producto1:
  RMSE Promedio: 22183.6933
  MAE Promedio: 22183.6933
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 248181.7601
  MAE Promedio: 248181.7601
  Número de iteraciones: 103


## Comparación de Resultados y Selección del Mejor Método de Validación

Comparamos los RMSE promedio de cada tipo de validación para seleccionar el mejor método.


In [12]:
# Crear DataFrame comparativo (solo Rolling Window y Expanding Window)
comparacion = []

for producto in df.columns:
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Rolling Window',
        'RMSE_Promedio': metricas_rw[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_rw[producto]['mae_promedio']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Expanding Window',
        'RMSE_Promedio': metricas_ew[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_ew[producto]['mae_promedio']
    })

df_comparacion = pd.DataFrame(comparacion)
print("\n" + "=" * 80)
print("COMPARACIÓN DE MÉTODOS DE VALIDACIÓN")
print("=" * 80)
print(df_comparacion.to_string(index=False))

# Identificar el mejor método para cada producto
print("\n" + "=" * 80)
print("MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)")
print("=" * 80)
mejor_validacion = {}
for producto in df.columns:
    producto_df = df_comparacion[df_comparacion['Producto'] == producto]
    mejor = producto_df.loc[producto_df['RMSE_Promedio'].idxmin()]
    mejor_validacion[producto] = mejor['Validacion']
    print(f"\n{producto}:")
    print(f"  Mejor método: {mejor['Validacion']}")
    print(f"  RMSE Promedio: {mejor['RMSE_Promedio']:.4f}")



COMPARACIÓN DE MÉTODOS DE VALIDACIÓN
 Producto       Validacion  RMSE_Promedio  MAE_Promedio
producto1   Rolling Window    2589.250700   2589.250700
producto1 Expanding Window   22183.693349  22183.693349
producto2   Rolling Window    6018.446793   6018.446793
producto2 Expanding Window  248181.760131 248181.760131

MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)

producto1:
  Mejor método: Rolling Window
  RMSE Promedio: 2589.2507

producto2:
  Mejor método: Rolling Window
  RMSE Promedio: 6018.4468


## Selección del Mejor Método de Validación y Modelo Final

Después de calcular las métricas promedio de cada tipo de validación, seleccionamos el mejor método basado en el RMSE promedio más bajo y entrenamos el modelo final para producción.


In [13]:
# Identificar los mejores parámetros para cada producto basado en RMSE promedio
print("\n" + "=" * 80)
print("SELECCIÓN DE PARÁMETROS PARA PRODUCCIÓN")
print("=" * 80)

mejores_parametros = {}

for producto in df.columns:
    producto_df = df_comparacion[df_comparacion['Producto'] == producto]
    mejor = producto_df.loc[producto_df['RMSE_Promedio'].idxmin()]
    
    # Obtener los parámetros usados en la validación inicial
    mejores_parametros[producto] = {
        'max_p': MAX_P_INIT,
        'max_d': MAX_D_INIT,
        'max_q': MAX_Q_INIT,
        'seasonal': SEASONAL_INIT,
        'rmse': mejor['RMSE_Promedio'],
        'metodo': mejor['Validacion']
    }
    
    print(f"\n{producto}:")
    print(f"  Mejor método de validación: {mejor['Validacion']}")
    print(f"  RMSE Promedio: {mejor['RMSE_Promedio']:.4f}")
    print(f"  MAE Promedio: {mejor['MAE_Promedio']:.4f}")
    print(f"  Parámetros Auto ARIMA: max_p={MAX_P_INIT}, max_d={MAX_D_INIT}, max_q={MAX_Q_INIT}")



SELECCIÓN DE PARÁMETROS PARA PRODUCCIÓN

producto1:
  Mejor método de validación: Rolling Window
  RMSE Promedio: 2589.2507
  MAE Promedio: 2589.2507
  Parámetros Auto ARIMA: max_p=5, max_d=3, max_q=5

producto2:
  Mejor método de validación: Rolling Window
  RMSE Promedio: 6018.4468
  MAE Promedio: 6018.4468
  Parámetros Auto ARIMA: max_p=5, max_d=3, max_q=5


In [14]:
# Los mejores parámetros ya fueron identificados en la sección anterior
# Usaremos los parámetros que dieron el mejor RMSE promedio


## Modelo Final para Producción

Entrenamos el modelo final con todos los datos disponibles usando los mejores parámetros encontrados (basados en el mejor RMSE promedio) y realizamos la predicción para el siguiente mes (1 pronóstico).


## Validación de Supuestos del Modelo Auto ARIMA

Después del entrenamiento del modelo final, es importante validar que se cumplan los supuestos estadísticos necesarios para que el modelo ARIMA sea válido:

1. **Estacionariedad**: Los residuales deben ser estacionarios
2. **Ruido blanco**: Los residuales deben comportarse como ruido blanco (sin autocorrelación)
3. **No autocorrelación**: No debe haber autocorrelación significativa en los errores
4. **No heterocedasticidad**: La varianza de los residuales debe ser constante
5. **Normalidad**: Los residuales deben seguir una distribución normal


In [15]:
def validar_supuestos_auto_arima(data, max_p=5, max_d=3, max_q=5, alpha=0.05):
    """
    Valida los supuestos estadísticos del modelo Auto ARIMA.
    
    Parameters:
    -----------
    data : array-like
        Serie temporal de datos
    max_p : int
        Valor máximo para el orden p (AR)
    max_d : int
        Valor máximo para el orden d (diferenciación)
    max_q : int
        Valor máximo para el orden q (MA)
    alpha : float
        Nivel de significancia para las pruebas (por defecto 0.05)
    
    Returns:
    --------
    dict
        Diccionario con los resultados de las validaciones:
        - 'estacionariedad': dict con resultados del test ADF
        - 'ruido_blanco': dict con resultados del test Ljung-Box
        - 'autocorrelacion': dict con resultados del test Ljung-Box
        - 'heterocedasticidad': dict con resultados del test ARCH
        - 'normalidad': dict con resultados del test Jarque-Bera
        - 'residuales': array con los residuales del modelo
        - 'modelo_ajustado': objeto del modelo ajustado
        - 'orden_encontrado': tuple con el orden (p, d, q) encontrado
    """
    resultados = {}
    
    # Convertir a pandas Series
    data_series = pd.Series(data)
    
    try:
        # Usar Auto ARIMA para encontrar el mejor modelo
        best_aic = np.inf
        best_order = None
        best_model = None
        
        # Buscar en el espacio de parámetros
        for p in range(0, max_p + 1):
            for d in range(0, max_d + 1):
                for q in range(0, max_q + 1):
                    try:
                        model = ARIMA(data_series, order=(p, d, q))
                        fitted_model = model.fit()
                        
                        if fitted_model.aic < best_aic:
                            best_aic = fitted_model.aic
                            best_order = (p, d, q)
                            best_model = fitted_model
                    except:
                        continue
        
        # Si no se encontró ningún modelo válido, usar un modelo simple
        if best_model is None:
            try:
                model = ARIMA(data_series, order=(1, 1, 1))
                best_model = model.fit()
                best_order = (1, 1, 1)
            except Exception as e:
                resultados['error'] = f"Error al ajustar el modelo: {e}"
                return resultados
        
        residuales = best_model.resid.dropna()
        resultados['modelo_ajustado'] = best_model
        resultados['residuales'] = residuales
        resultados['orden_encontrado'] = best_order
        
        # 1. ESTACIONARIEDAD (Test de Dickey-Fuller Aumentado)
        try:
            adf_result = adfuller(residuales, autolag='AIC')
            adf_statistic = adf_result[0]
            adf_pvalue = adf_result[1]
            es_estacionario = adf_pvalue < alpha
            
            resultados['estacionariedad'] = {
                'estadistico': adf_statistic,
                'pvalor': adf_pvalue,
                'se_cumple': es_estacionario,
                'interpretacion': 'Estacionario' if es_estacionario else 'No estacionario'
            }
        except Exception as e:
            resultados['estacionariedad'] = {
                'estadistico': np.nan,
                'pvalor': np.nan,
                'se_cumple': False,
                'interpretacion': f'Error: {str(e)}'
            }
        
        # 2. RUIDO BLANCO Y AUTOCORRELACIÓN (Test de Ljung-Box)
        try:
            max_lag = min(10, int(np.sqrt(len(residuales))))
            ljung_box = acorr_ljungbox(residuales, lags=max_lag, return_df=True)
            
            lb_pvalue = ljung_box['lb_pvalue'].iloc[-1]
            lb_statistic = ljung_box['lb_stat'].iloc[-1]
            es_ruido_blanco = lb_pvalue > alpha
            
            resultados['ruido_blanco'] = {
                'estadistico': lb_statistic,
                'pvalor': lb_pvalue,
                'se_cumple': es_ruido_blanco,
                'interpretacion': 'Ruido blanco' if es_ruido_blanco else 'No es ruido blanco'
            }
            
            resultados['autocorrelacion'] = {
                'estadistico': lb_statistic,
                'pvalor': lb_pvalue,
                'se_cumple': es_ruido_blanco,
                'interpretacion': 'Sin autocorrelación' if es_ruido_blanco else 'Autocorrelación presente'
            }
        except Exception as e:
            resultados['ruido_blanco'] = {
                'estadistico': np.nan,
                'pvalor': np.nan,
                'se_cumple': False,
                'interpretacion': f'Error: {str(e)}'
            }
            resultados['autocorrelacion'] = {
                'estadistico': np.nan,
                'pvalor': np.nan,
                'se_cumple': False,
                'interpretacion': f'Error: {str(e)}'
            }
        
        # 3. HETEROCEDASTICIDAD (Test ARCH)
        try:
            arch_test = het_arch(residuales, maxlag=min(5, len(residuales)//10))
            arch_statistic = arch_test[0]
            arch_pvalue = arch_test[1]
            no_heterocedasticidad = arch_pvalue > alpha
            
            resultados['heterocedasticidad'] = {
                'estadistico': arch_statistic,
                'pvalor': arch_pvalue,
                'se_cumple': no_heterocedasticidad,
                'interpretacion': 'Homocedástico' if no_heterocedasticidad else 'Heterocedástico'
            }
        except Exception as e:
            resultados['heterocedasticidad'] = {
                'estadistico': np.nan,
                'pvalor': np.nan,
                'se_cumple': False,
                'interpretacion': f'Error: {str(e)}'
            }
        
        # 4. NORMALIDAD (Test de Jarque-Bera)
        try:
            jb_statistic, jb_pvalue = stats.jarque_bera(residuales)
            es_normal = jb_pvalue > alpha
            
            resultados['normalidad'] = {
                'estadistico': jb_statistic,
                'pvalor': jb_pvalue,
                'se_cumple': es_normal,
                'interpretacion': 'Normal' if es_normal else 'No normal'
            }
        except Exception as e:
            resultados['normalidad'] = {
                'estadistico': np.nan,
                'pvalor': np.nan,
                'se_cumple': False,
                'interpretacion': f'Error: {str(e)}'
            }
        
    except Exception as e:
        print(f"Error al ajustar el modelo: {e}")
        resultados['error'] = str(e)
    
    return resultados


## Validación de Supuestos del Modelo Final

A continuación se validan los supuestos estadísticos del modelo Auto ARIMA final para cada producto.


In [16]:
# Validar supuestos para cada producto
print("\n" + "=" * 80)
print("VALIDACIÓN DE SUPUESTOS DEL MODELO AUTO ARIMA")
print("=" * 80)

resultados_validacion = {}
tabla_resumen = []

for producto in df.columns:
    print(f"\n{'='*80}")
    print(f"VALIDACIÓN DE SUPUESTOS - {producto.upper()}")
    print(f"{'='*80}")
    
    data = df[producto].values
    params = mejores_parametros[producto]
    
    print(f"\nParámetros Auto ARIMA: max_p={params['max_p']}, max_d={params['max_d']}, max_q={params['max_q']}")
    
    # Realizar validación de supuestos
    validacion = validar_supuestos_auto_arima(
        data, 
        max_p=params['max_p'],
        max_d=params['max_d'],
        max_q=params['max_q'],
        alpha=0.05
    )
    resultados_validacion[producto] = validacion
    
    # Mostrar resultados detallados
    if 'error' not in validacion:
        if 'orden_encontrado' in validacion:
            print(f"Orden encontrado por Auto ARIMA: ARIMA{validacion['orden_encontrado']}")
        
        print("\n--- Resultados de las Pruebas ---")
        
        # Estacionariedad
        est = validacion['estacionariedad']
        print(f"\n1. ESTACIONARIEDAD (Test ADF):")
        print(f"   Estadístico: {est['estadistico']:.4f}")
        print(f"   p-valor: {est['pvalor']:.4f}")
        print(f"   Resultado: {est['interpretacion']}")
        
        # Ruido blanco
        rb = validacion['ruido_blanco']
        print(f"\n2. RUIDO BLANCO (Test Ljung-Box):")
        print(f"   Estadístico: {rb['estadistico']:.4f}")
        print(f"   p-valor: {rb['pvalor']:.4f}")
        print(f"   Resultado: {rb['interpretacion']}")
        
        # Autocorrelación
        ac = validacion['autocorrelacion']
        print(f"\n3. AUTOCORRELACIÓN (Test Ljung-Box):")
        print(f"   Estadístico: {ac['estadistico']:.4f}")
        print(f"   p-valor: {ac['pvalor']:.4f}")
        print(f"   Resultado: {ac['interpretacion']}")
        
        # Heterocedasticidad
        het = validacion['heterocedasticidad']
        print(f"\n4. HETEROCEDASTICIDAD (Test ARCH):")
        print(f"   Estadístico: {het['estadistico']:.4f}")
        print(f"   p-valor: {het['pvalor']:.4f}")
        print(f"   Resultado: {het['interpretacion']}")
        
        # Normalidad
        norm = validacion['normalidad']
        print(f"\n5. NORMALIDAD (Test Jarque-Bera):")
        print(f"   Estadístico: {norm['estadistico']:.4f}")
        print(f"   p-valor: {norm['pvalor']:.4f}")
        print(f"   Resultado: {norm['interpretacion']}")
        
        # Agregar a tabla resumen
        tabla_resumen.append({
            'Producto': producto,
            'Supuesto': 'Estacionariedad',
            'Se Cumple': 'Sí' if est['se_cumple'] else 'No',
            'p-valor': f"{est['pvalor']:.4f}",
            'Interpretación': est['interpretacion']
        })
        tabla_resumen.append({
            'Producto': producto,
            'Supuesto': 'Ruido Blanco',
            'Se Cumple': 'Sí' if rb['se_cumple'] else 'No',
            'p-valor': f"{rb['pvalor']:.4f}",
            'Interpretación': rb['interpretacion']
        })
        tabla_resumen.append({
            'Producto': producto,
            'Supuesto': 'No Autocorrelación',
            'Se Cumple': 'Sí' if ac['se_cumple'] else 'No',
            'p-valor': f"{ac['pvalor']:.4f}",
            'Interpretación': ac['interpretacion']
        })
        tabla_resumen.append({
            'Producto': producto,
            'Supuesto': 'No Heterocedasticidad',
            'Se Cumple': 'Sí' if het['se_cumple'] else 'No',
            'p-valor': f"{het['pvalor']:.4f}",
            'Interpretación': het['interpretacion']
        })
        tabla_resumen.append({
            'Producto': producto,
            'Supuesto': 'Normalidad',
            'Se Cumple': 'Sí' if norm['se_cumple'] else 'No',
            'p-valor': f"{norm['pvalor']:.4f}",
            'Interpretación': norm['interpretacion']
        })
    else:
        print(f"\nError en la validación: {validacion['error']}")

# Crear DataFrame con tabla resumen
df_resumen_supuestos = pd.DataFrame(tabla_resumen)

# Mostrar tabla resumen
print("\n" + "=" * 80)
print("TABLA RESUMEN DE VALIDACIÓN DE SUPUESTOS")
print("=" * 80)
print(df_resumen_supuestos.to_string(index=False))

# Determinar si se puede continuar con producción
print("\n" + "=" * 80)
print("EVALUACIÓN FINAL - ¿CONTINUAR CON PRODUCCIÓN?")
print("=" * 80)

for producto in df.columns:
    if producto in resultados_validacion and 'error' not in resultados_validacion[producto]:
        validacion = resultados_validacion[producto]
        
        # Contar cuántos supuestos se cumplen
        supuestos_cumplidos = sum([
            validacion['estacionariedad']['se_cumple'],
            validacion['ruido_blanco']['se_cumple'],
            validacion['autocorrelacion']['se_cumple'],
            validacion['heterocedasticidad']['se_cumple'],
            validacion['normalidad']['se_cumple']
        ])
        
        total_supuestos = 5
        porcentaje_cumplimiento = (supuestos_cumplidos / total_supuestos) * 100
        
        # Criterio: al menos 3 de 5 supuestos deben cumplirse
        # Los más críticos son: estacionariedad, ruido blanco y no autocorrelación
        supuestos_criticos = [
            validacion['estacionariedad']['se_cumple'],
            validacion['ruido_blanco']['se_cumple'],
            validacion['autocorrelacion']['se_cumple']
        ]
        criticos_cumplidos = sum(supuestos_criticos)
        
        puede_produccion = criticos_cumplidos >= 2 and supuestos_cumplidos >= 3
        
        print(f"\n{producto}:")
        print(f"  Supuestos cumplidos: {supuestos_cumplidos}/{total_supuestos} ({porcentaje_cumplimiento:.1f}%)")
        print(f"  Supuestos críticos cumplidos: {criticos_cumplidos}/3")
        print(f"  Decisión: {'✓ APROBADO PARA PRODUCCIÓN' if puede_produccion else '✗ NO APROBADO - Revisar modelo'}")
        
        if not puede_produccion:
            print(f"  Recomendación: Revisar el modelo o considerar transformaciones adicionales")
    else:
        print(f"\n{producto}:")
        print(f"  Error en la validación - No se puede evaluar")



VALIDACIÓN DE SUPUESTOS DEL MODELO AUTO ARIMA

VALIDACIÓN DE SUPUESTOS - PRODUCTO1

Parámetros Auto ARIMA: max_p=5, max_d=3, max_q=5
Orden encontrado por Auto ARIMA: ARIMA(4, 2, 5)

--- Resultados de las Pruebas ---

1. ESTACIONARIEDAD (Test ADF):
   Estadístico: -56.9821
   p-valor: 0.0000
   Resultado: Estacionario

2. RUIDO BLANCO (Test Ljung-Box):
   Estadístico: 19.2665
   p-valor: 0.0370
   Resultado: No es ruido blanco

3. AUTOCORRELACIÓN (Test Ljung-Box):
   Estadístico: 19.2665
   p-valor: 0.0370
   Resultado: Autocorrelación presente

4. HETEROCEDASTICIDAD (Test ARCH):
   Estadístico: 3.0272
   p-valor: 0.6958
   Resultado: Homocedástico

5. NORMALIDAD (Test Jarque-Bera):
   Estadístico: 33703.6189
   p-valor: 0.0000
   Resultado: No normal

VALIDACIÓN DE SUPUESTOS - PRODUCTO2

Parámetros Auto ARIMA: max_p=5, max_d=3, max_q=5
Orden encontrado por Auto ARIMA: ARIMA(2, 2, 1)

--- Resultados de las Pruebas ---

1. ESTACIONARIEDAD (Test ADF):
   Estadístico: -29.5802
   p-valor:

In [17]:
# Entrenar modelo final y realizar predicción para producción
print("\n" + "=" * 80)
print("PREDICCIÓN FINAL PARA PRODUCCIÓN")
print("=" * 80)

predicciones_produccion = {}

for producto in df.columns:
    print(f"\n--- {producto} ---")
    data = df[producto].values
    
    # Obtener mejores parámetros
    params = mejores_parametros[producto]
    
    print(f"Parámetros del modelo Auto ARIMA:")
    print(f"  max_p: {params['max_p']}")
    print(f"  max_d: {params['max_d']}")
    print(f"  max_q: {params['max_q']}")
    print(f"  seasonal: {params['seasonal']}")
    print(f"  RMSE en validación: {params['rmse']:.4f}")
    print(f"  Método de validación usado: {params['metodo']}")
    
    # Entrenar modelo final con todos los datos usando Auto ARIMA
    try:
        # Usar Auto ARIMA para encontrar el mejor modelo y realizar predicción
        prediccion = auto_arima_forecast(
            data,
            max_p=params['max_p'],
            max_d=params['max_d'],
            max_q=params['max_q'],
            seasonal=params['seasonal'],
            forecast_horizon=1,
            suppress_warnings=True
        )
        
        predicciones_produccion[producto] = prediccion
        
        print(f"  Predicción para el siguiente mes: {prediccion:.4f}")
        print(f"  Último valor observado: {data[-1]:.4f}")
        print(f"  Diferencia: {prediccion - data[-1]:.4f}")
        
    except Exception as e:
        print(f"  Error al generar predicción: {e}")
        # Fallback: usar último valor
        predicciones_produccion[producto] = data[-1]
        print(f"  Predicción (fallback): {data[-1]:.4f}")

# Mostrar resumen de predicciones
print("\n" + "=" * 80)
print("RESUMEN DE PREDICCIONES PARA PRODUCCIÓN")
print("=" * 80)
for producto, prediccion in predicciones_produccion.items():
    print(f"{producto}: {prediccion:.4f}")



PREDICCIÓN FINAL PARA PRODUCCIÓN

--- producto1 ---
Parámetros del modelo Auto ARIMA:
  max_p: 5
  max_d: 3
  max_q: 5
  seasonal: False
  RMSE en validación: 2589.2507
  Método de validación usado: Rolling Window
  Predicción para el siguiente mes: 128.8965
  Último valor observado: 141.9909
  Diferencia: -13.0944

--- producto2 ---
Parámetros del modelo Auto ARIMA:
  max_p: 5
  max_d: 3
  max_q: 5
  seasonal: False
  RMSE en validación: 6018.4468
  Método de validación usado: Rolling Window
  Predicción para el siguiente mes: 691.3015
  Último valor observado: 676.0581
  Diferencia: 15.2434

RESUMEN DE PREDICCIONES PARA PRODUCCIÓN
producto1: 128.8965
producto2: 691.3015
